In [1]:
# =========================
# Setup
# =========================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import random

# For reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# =========================
# Gaussian Noise Layer
# =========================

class GaussianNoise(nn.Module):
    def __init__(self, mean=0.0, std=0.2):
        super().__init__()
        self.mean = mean
        self.std = std

    def forward(self, x):
        if self.training:
            return x + torch.randn_like(x) * self.std + self.mean
        return x

In [3]:
class ClassificationModel(nn.Module):
    def __init__(self, num_features=44, num_classes=6):
        super().__init__()
        self.model = nn.Sequential(
            nn.BatchNorm1d(num_features),
            GaussianNoise(0.0, 0.2),
            nn.Linear(num_features, 1000),
            nn.ReLU(),

            nn.BatchNorm1d(1000),
            nn.Dropout(0.2),
            GaussianNoise(0.0, 0.2),
            nn.Linear(1000, 200),
            nn.ReLU(),

            nn.BatchNorm1d(200),
            nn.Dropout(0.2),
            GaussianNoise(0.0, 0.2),
            nn.Linear(200, 200),
            nn.ReLU(),

            nn.BatchNorm1d(200),
            nn.Dropout(0.2),
            GaussianNoise(0.0, 0.2),
            nn.Linear(200, 200),
            nn.ReLU(),

            nn.BatchNorm1d(200),
            nn.Dropout(0.2),
            nn.Linear(200, 200),
            nn.ReLU(),

            nn.Linear(200, num_classes)
        )
        self._init_weights()

    def _init_weights(self):
        for m in self.model:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.model(x)

In [4]:
# =========================
# Training & Evaluation
# =========================

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

from sklearn.metrics import confusion_matrix

def evaluate(model, loader, criterion):
    model.eval()
    y_true, y_pred = [], []
    val_loss = 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            val_loss += criterion(outputs, y).item()
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="weighted")

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # For binary classification
    if cm.shape == (2, 2):
        TN, FP, FN, TP = cm.ravel()
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    else:
        # For multiclass, you might want per-class sensitivity and specificity,
        # or macro-averaged values (more complex, depends on your task)
        sensitivity, specificity = None, None

    return val_loss / len(loader), acc, f1, sensitivity, specificity

In [5]:
mwr_df_simple = pd.read_csv('mwr_simple.csv')

In [6]:
# Select feature columns (sensor readings)
feature_cols = [col for col in mwr_df_simple.columns if col.endswith('int') or col.endswith('sk')]
mwr_df_simple['features'] = mwr_df_simple[feature_cols].values.tolist()

# Prepare labels and text targets
mwr_df_simple['class_label'] = mwr_df_simple['y_binary'].astype(int)

In [10]:
train_df, val_df = train_test_split(mwr_df_simple, test_size=0.2, random_state=42)

In [8]:
# =========================
# Dataset and DataLoader
# =========================

class TabularDataset(Dataset):
    def __init__(self, df):
        self.X = torch.tensor(df['features'].tolist(), dtype=torch.float32)
        self.y = torch.tensor(df['class_label'].tolist(), dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TabularDataset(train_df)
val_dataset = TabularDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [9]:
# =========================
# Run Training Loop
# =========================

model = ClassificationModel(num_features=44, num_classes=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

epochs = 30
for epoch in range(1, epochs + 1):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, acc, f1, sensitivity, specificity = evaluate(model, val_loader, criterion)
    print(
        f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
        f"Acc: {acc:.4f} | F1: {f1:.4f} | Sensitivity: {sensitivity:.4f} | Specificity: {specificity:.4f}"
    )

Epoch 01 | Train Loss: 0.6835 | Val Loss: 0.5101 | Acc: 0.7599 | F1: 0.6829 | Sensitivity: 0.9778 | Specificity: 0.0648
Epoch 02 | Train Loss: 0.5404 | Val Loss: 0.4987 | Acc: 0.7632 | F1: 0.7390 | Sensitivity: 0.9103 | Specificity: 0.2939
Epoch 03 | Train Loss: 0.5180 | Val Loss: 0.4976 | Acc: 0.7572 | F1: 0.7441 | Sensitivity: 0.8805 | Specificity: 0.3639
Epoch 04 | Train Loss: 0.5141 | Val Loss: 0.4893 | Acc: 0.7721 | F1: 0.7489 | Sensitivity: 0.9160 | Specificity: 0.3129
Epoch 05 | Train Loss: 0.4994 | Val Loss: 0.4777 | Acc: 0.7719 | F1: 0.7618 | Sensitivity: 0.8840 | Specificity: 0.4140
Epoch 06 | Train Loss: 0.4954 | Val Loss: 0.4879 | Acc: 0.7568 | F1: 0.7519 | Sensitivity: 0.8572 | Specificity: 0.4365
Epoch 07 | Train Loss: 0.4900 | Val Loss: 0.4807 | Acc: 0.7642 | F1: 0.7589 | Sensitivity: 0.8640 | Specificity: 0.4460
Epoch 08 | Train Loss: 0.4877 | Val Loss: 0.4769 | Acc: 0.7628 | F1: 0.7580 | Sensitivity: 0.8610 | Specificity: 0.4494
Epoch 09 | Train Loss: 0.4773 | Val Loss